<a href="https://colab.research.google.com/github/bkricardo/music_genretion_project/blob/main/lstm_modified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#using google colab to read data from google drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
import glob
import pickle
import numpy
from music21 import *
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [ ]:
def train_network():
    """ Train a Neural Network to generate music """
    notes = get_notes()

    # get amount of pitch names
    n_vocab = len(set(notes))

    network_input, network_output = prepare_sequences(notes, n_vocab)

    model = create_network(network_input, n_vocab)

    global history

    history = train(model, network_input, network_output)


In [3]:
def get_notes():
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = []

    for file in glob.glob("drive/My Drive/music/test2/*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None
        """ maybe we can add a filter to keep only midi with time signiture 2/4 or 4/4"""

        try: # file has instrument parts
        
            #directly read the midi melody at part 0
            notes_to_parse = midi[0].notes
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                #add duration
                notes.append(str(element.pitch)+"="+str(element.duration.quarterLength))
            elif isinstance(element, chord.Chord):
                #add duration
                notes.append('&'.join(str(n) for n in element.normalOrder)+ "=" + str(element.duration.quarterLength))

    with open('drive/My Drive/music/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes

In [ ]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

In [ ]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

In [ ]:
def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "drive/My Drive/music/weights.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    f = model.fit(network_input, network_output, epochs=10, batch_size=256, callbacks=callbacks_list)

    return f

In [ ]:
if __name__ == '__main__':
    train_network()

Parsing drive/My Drive/music/test2/cs1-1pre.mid
Parsing drive/My Drive/music/test2/cs1-2all.mid
Parsing drive/My Drive/music/test2/cs1-3cou.mid
Parsing drive/My Drive/music/test2/cs1-4sar.mid
Parsing drive/My Drive/music/test2/cs1-5men.mid
Epoch 1/10
13/13 [==============================] - 9s 686ms/step - loss: 5.2865
Epoch 2/10
13/13 [==============================] - 8s 632ms/step - loss: 5.3233
Epoch 3/10
13/13 [==============================] - 9s 675ms/step - loss: 5.2535
Epoch 4/10
13/13 [==============================] - 8s 640ms/step - loss: 5.0632
Epoch 5/10
13/13 [==============================] - 9s 698ms/step - loss: 4.9833
Epoch 6/10
13/13 [==============================] - 9s 670ms/step - loss: 4.8299
Epoch 7/10
13/13 [==============================] - 9s 661ms/step - loss: 4.6983
Epoch 8/10
13/13 [==============================] - 8s 640ms/step - loss: 4.5881
Epoch 9/10
13/13 [==============================] - 9s 674ms/step - loss: 4.3986
Epoch 10/10
13/13 [============